In [13]:
#Necessary Imports for Data Cleaning and Analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [15]:
job_desc_df = pd.read_csv('fake_job_postings.csv')

In [17]:
job_desc_df

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,"CA, ON, Toronto",Sales,NaN,Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0,1,1,Full-time,Mid-Senior level,NaN,Computer Software,Sales,0
17876,17877,Payroll Accountant,"US, PA, Philadelphia",Accounting,NaN,WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing,0
17877,17878,Project Cost Control Staff Engineer - Cost Con...,"US, TX, Houston",NaN,NaN,We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,NaN,0,0,0,Full-time,NaN,NaN,NaN,NaN,0
17878,17879,Graphic Designer,"NG, LA, Lagos",NaN,NaN,NaN,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0,0,1,Contract,Not Applicable,Professional,Graphic Design,Design,0


In [19]:
(job_desc_df.isna().sum() / len(job_desc_df)) * 100

job_id                  0.000000
title                   0.000000
location                1.935123
department             64.580537
salary_range           83.959732
company_profile        18.501119
description             0.005593
requirements           15.078300
benefits               40.335570
telecommuting           0.000000
has_company_logo        0.000000
has_questions           0.000000
employment_type        19.412752
required_experience    39.429530
required_education     45.329978
industry               27.421700
function               36.101790
fraudulent              0.000000
dtype: float64

In [21]:
#Changing `Nan` values and other values indicating a lack of information with the 'Not Provided' string
job_desc_df['department'] = job_desc_df['department'].replace(['Unknown', 'nan', '...', '0', ' '], np.nan)
job_desc_df['department'] = job_desc_df['department'].fillna('Not Provided')

#Sanity Check
assert job_desc_df['department'].isna().sum() == 0, 'NaN values in `department` column'

In [23]:
department_mapping = {
    "Marketing": ["Marketing", "MKT", "MKTG", "Marketing and Communications", "Marketing – Pame Stoixima",
                  "Marketing & Sales", "Marketing / Customer Service", "Marketing Department", "Marketing team",
                  "Marketing & Design", "Marketing, Sales", "Marketing Intern"],

    "Sales": ["Sales", "Sales and Business Development", "Sales and Marketing", "Sales & Marketing",
              "Sales Team", "Inside Sales", "Inside Sales Team", "Local Sales", "New Business Sales",
              "Field Sales", "Sales - USA", "Sales Department", "Sales / BD", "Sales / Marketing",
              "Sales&Support", "Sales/Marketing", "Sales / Engineering", "Outbound Sales"],

    "Human Resources": ["HR", "Human Resources", "Recruiting HR", "HR Manager", "Recruitment",
                        "Recruitment ", "People & Culture", "Organisational Development"],

    "Customer Service": ["Customer Service", "Customer Care", "Servicing-Customer Care",
                         "Customer Support", "Customer and Compliance", "Client Services",
                         "Member Services", "Client Success", "CustomerSuccess", "Customer Experience",
                         "Customer Service Rep.", "Customer Champion", "Customer Happiness"],

    "Engineering": ["Engineering", "Engineering ", "Engineeering", "Engineering:Software Engineering",
                    "Engineering:Dev Ops", "Software Engineering", "Electrical", "Engineers",
                    "Engineering - Construction", "Engineering, Pipeline Design", "Design Engineering"],

     "Technology": ["Software Development", "Software Engineering", "Application Development",
                    "Front-End Development", "Back-End Development", "Web Development",
                    "Mobile Development", "Game Design and Development", "Software / IT",
                    "Tech", "Programming", "Engineering: Software Engineering", "Engineering: Dev Ops",
                    "Development", "Technology", "Digital Design", "UI/UX Design",
                    "UX Practice", "UX/API", "Digital Marketing", "Digital Department",
                    "Dev and Software QA", "Dev - Web Team"],


    "IT": ["IT", "IT Department", "IT Team", "IT Solution", "IT Sales",
           "Information Technology", "Information Technology Group",
           "Computer / Software", "IT Support", "IT Services", "IT Operations",
           "Server Administration", "Infrastructure & Operations", "Infrastructure Support Services",
           "Network Operations Center", "System Operations", "Technical Support",
           "Security", "Cybersecurity", "Cloud Services", "CloudSpotter Technologies",
           "Admin/Clerical", "Help Desk"],

    "Operations": ["Operations", "Operations ", "Infrastructure & Operations",
                   "Infrastructure Support Services", "Field Operations", "Online Operations",
                   "Business Operations", "Businessfriend.com", "Platform", "Business Development",
                   "Strategic Initiatives", "Facility", "Facilities / Maintenance"],

    "Finance & Accounting": ["Finance", "Financial", "Finance/Securities", "Finance and Operations",
                              "Accounting", "Accounting / Finance", "Accounting & Finance",
                              "Finance & Accounting", "Financial Services"],

    "Product Management & Development": ["Product", "Product Management", "Product Team",
                                         "Product Marketing", "Product & Innovation",
                                         "Product Development", "Product Development Team",
                                         "Product Innovation", "Product Development - Test"],

    "Research & Development": ["R&D", "R&D Department", "Research", "Research & Development",
                                "Development", "Development ", "Development Dpt", "Development & Support"],

    "Manufacturing & Production": ["Manufacturing", "Manufacturing & Production", "Production",
                                    "Production ", "Manufacturing - Quality"],

    "Legal & Compliance": ["Legal", "Legal Services", "Finance, legal & compliance",
                            "Compliance", "RISK MANAGEMENT ", "Contracts", "AML", "FRAUD DEPT"],

    "Administrative": ["Admin", "Administration", "Administrative", "Administrative Office",
                       "Administration ", "Management", "Management Team", "Upper Level Management",
                       "Lower Level Management", "Entry Level Supervision"],

    "Healthcare": ["Medical", "Healthcare", "Health Services", "Health and Social", "Health and Social ",
                   "Family Medicine", "Nursing", "Physical Therapy", "Healthcare Industries"],

    "Creative & Design": ["Creative", "Creative Services", "Creative Department", "Design",
                          "Designing", "Graphic Design", "Digital Design", "Design & Interface",
                          "Design & Marketing", "Design/Architecture"],

    "Media & Communications": ["Media", "Social Media", "Social Media Promotion",
                                "Social Media Services", "Public Relations", "PR",
                                "Public Relations & Communication", "Marketing & Communications",
                                "Content Programming", "Broadcasting"],

    "Project Management": ["Project Management", "Project Management ", "Projects",
                            "Program", "Programs", "Campaigns", "Partnerships"],

    "Retail & Consumer": ["Retail", "Retail Pharmacy", "Grocery Stores", "Merchandising",
                           "Merchandising ", "Consumer", "Auto Sales"],

    "Business Consulting": ["Consulting", "Professional Services", "Professional Services Team",
                             "Business Consultancy", "Advisory", "Business Intelligence"],

    "Education & Training": ["Education", "Instructor", "Academic", "Student Financial Services",
                              "Community Training Center", "Training"],

    "Logistics & Supply Chain": ["Logistics", "Supply", "Supply & Chain", "Transport",
                                  "Shipping", "Warehouse", "Warehouse "],

    "Hospitality & Food Services": ["Catering", "Food", "Food and Beverage", "Restaurant",
                                     "Hotel", "Hospitality"],

    "Quality Assurance & Testing": ["Quality Assurance", "Quality Engineering", "QA",
                                     "Testing", "Solution Test", "Integration & Testing"],

    "Government & Public Sector": ["Government", "Public Affairs", "Administration support",
                                    "Head Office", "State of Indiana"]
}


In [25]:
def standardize_department(department_name):
    for standard, variations in department_mapping.items():
        if department_name in variations:
            return standard
    return 'Not Provided'  # Default if no match is found

job_desc_df['Standardized_Department'] = job_desc_df['department'].apply(standardize_department)

In [27]:
#Changing `NaN` values with the 'Not Provided' string
job_desc_df['company_profile'] = job_desc_df['company_profile'].fillna('Not Provided')

#Sanity Check
assert job_desc_df['company_profile'].isna().sum() == 0, 'NaN values in `company_profile` column'

In [29]:
#Function to extract and clean salary range column
def extract_salary(salary_str):
  if pd.isna(salary_str):
    return np.nan, np.nan
  salary_str = salary_str.replace('$', '').replace(',', '').lower()

  if '/hour' in salary_str or 'hourly' in salary_str:
    try:
      hourly_rate = float(re.findall(r'\d+', salary_str)[0])
      annual_salary = hourly_rate * 40 * 52
      return int(annual_salary * 0.9), int(annual_salary * 1.1)
    except:
      return np.nan, np.nan

  if '-' in salary_str:
    try:
      min_salary, max_salary = salary_str.split('-')
      return int(min_salary.strip()), int(max_salary.strip())
    except:
      return np.nan, np.nan

#Apply function to dataset
job_desc_df[['min_salary', 'max_salary']] = job_desc_df['salary_range'].apply(lambda x: pd.Series(extract_salary(x)))

# Fill NaN values with "Not Provided"
job_desc_df[['min_salary', 'max_salary']] = job_desc_df[['min_salary', 'max_salary']].fillna(0)

# Calculate salary range difference
job_desc_df['salary_diff'] = job_desc_df['max_salary'] - job_desc_df['min_salary']

In [30]:
#Changing 'NaN' values to the 'Not Provided' string
job_desc_df['requirements'] = job_desc_df['requirements'].fillna('Not Provided')

#Sanity Check
assert job_desc_df['requirements'].isna().sum() == 0, 'NaN values in `requirements` column'

In [33]:
#Changing 'NaN' and other values indicating a lack of information to the 'Not Provided' string
job_desc_df['benefits'] = job_desc_df['benefits'].replace(['nan', '...'], np.nan)
job_desc_df['benefits'] = job_desc_df['benefits'].fillna('Not Provided')

#Sanity Check
assert job_desc_df['benefits'].isna().sum() == 0, 'NaN values in `benefits` column'

In [35]:
#Chaning `NaN` values to 'Not Provided'
job_desc_df['employment_type'] = job_desc_df['employment_type'].replace(['nan'], np.nan)
job_desc_df['employment_type'] = job_desc_df['employment_type'].fillna('Not Provided')

#Sanity Check
assert job_desc_df['employment_type'].isna().sum() == 0, 'NaN values in `employment_type` column'

In [37]:
#Changing values
job_desc_df['required_experience'] = job_desc_df['required_experience'].replace(['nan'], np.nan)
job_desc_df['required_experience'] = job_desc_df['required_experience'].fillna('Not Provided')
#Sanity Check
assert job_desc_df['required_experience'].isna().sum() == 0, 'NaN values in `required_experience` column'

In [39]:
#Changing values
job_desc_df['required_education'] = job_desc_df['required_education'].replace(['nan'], np.nan)
job_desc_df['required_education'] = job_desc_df['required_education'].fillna('Not Provided')

#Sanity Check
assert job_desc_df['required_education'].isna().sum() == 0, 'NaN values in `required_education` column'

In [41]:
#Changing values
job_desc_df['industry'] = job_desc_df['industry'].replace(['nan', '...'], np.nan)
job_desc_df['industry'] = job_desc_df['industry'].fillna('Not Provided')

#Sanity Check
assert job_desc_df['industry'].isna().sum() == 0, 'NaN values in `industry` column'

In [43]:
industry_mapping = {
    'Marketing': ['Marketing and Advertising', 'Public Relations and Communications', 'Market Research'],
    'Electronics': ['Computer Software', 'Consumer Electronics', 'Electrical/Electronic Manufacturing', 'Computer Hardware'],
    'Health Care': ['Hospital and Health Care', 'Medical Practice', 'Pharmaceuticals', 'Mental Health Care', 'Medical Devices'],
    'Finance': ['Financial Services', 'Banking', 'Venture Capital & Private Equity', 'Investment Banking', 'Investment Management', 'Accounting'],
    'Education': ['Education Management', 'E-Learning', 'Primary/Secondary Education', 'Higher Education'],
    'IT Services': ['Information Technology and Services', 'Computer Networking', 'Computer & Network Security', 'Information Services'],
    'Media': ['Online Media', 'Media Production', 'Broadcast Media'],
    'Consumer Goods': ['Cosmetics', 'Luxury Goods & Jewelry', 'Apparel & Fashion', 'Sporting Goods'],
    'Automotive': ['Automotive'],
    'Law': ['Legal Services', 'Law Practice'],
    'Engineering': ['Mechanical or Industrial Engineering', 'Civil Engineering'],
    'Construction': ['Building Materials', 'Construction'],
    'Retail': ['Retail'],
    'Agriculture': ['Farming', 'Ranching'],
    'Real Estate': ['Real Estate', 'Commercial Real Estate'],
    'Government': ['Government Administration', 'Government Relations'],
    'Entertainment': ['Music', 'Motion Pictures and Film', 'Performing Arts'],
    'Transportation': ['Transportation/Trucking/Railroad', 'Airlines/Aviation', 'Aviation & Aerospace'],
    'Security': ['Security and Investigations', 'Law Enforcement'],
    'Environmental Services': ['Renewables & Environment', 'Environmental Services'],
    'Consulting': ['Management Consulting'],
    'Nonprofit': ['Nonprofit Organization Management', 'Philanthropy'],
    'Not Provided' : ['Other']
}

def standardize_industry(industry_name):
    for standard, variations in industry_mapping.items():
        if industry_name in variations:
            return standard
    return 'Not Provided'  # Default if no match is found

job_desc_df['Standardized_Industry'] = job_desc_df['industry'].apply(standardize_industry)

In [45]:
#Changing values
job_desc_df['function'] = job_desc_df['function'].replace(['nan'], np.nan)
job_desc_df['function'] = job_desc_df['function'].fillna('Not Provided')

#Sanity Check
assert job_desc_df['function'].isna().sum() == 0, 'NaN values in `function` column'

In [47]:
#Saving the cleaned data and new column(s) to a new file path and name
job_desc_df.to_csv('cleaned_fake_job_postings.csv', index=False)

In [49]:
from sklearn.preprocessing import OrdinalEncoder

experience_order = [
    'Not Provided' , # ADDED
    'Not Applicable',
    'Internship',
    'Entry level',
    'Associate',
    'Mid-Senior level',
    'Director',
    'Executive'
]

encoder = OrdinalEncoder(categories=[experience_order])
job_desc_df['experience_encoded'] = encoder.fit_transform(job_desc_df[['required_experience']]).astype(int)


In [51]:
job_desc_df['job_desc_length'] = job_desc_df['description'].str.len()

In [53]:
from sklearn.preprocessing import OrdinalEncoder

education_order = [
    'Not Provided', 'Unspecified', 'Some High School Coursework', 'High School or equivalent',
    'Vocational - HS Diploma', 'Some College Coursework Completed', 'Certification',
    'Vocational', 'Vocational - Degree', 'Associate Degree', "Bachelor's Degree",
    "Master's Degree", 'Doctorate', 'Professional'
]

encoder = OrdinalEncoder(categories=[education_order])
job_desc_df['education_encoded'] = encoder.fit_transform(job_desc_df[['required_education']]).astype(int)

In [55]:
employment_order = ['Not Provided', 'Other', 'Temporary', 'Contract',
                    'Part-time', 'Full-time']

encoder = OrdinalEncoder(categories=[employment_order])
job_desc_df['employment_encoded'] = encoder.fit_transform(job_desc_df[['employment_type']]).astype(int)

In [57]:
from sklearn.preprocessing import OneHotEncoder

#parse_out='False' returns a dense MumPy array
encoder = OneHotEncoder(sparse_output=False)

#Fit and transform the 'standardized_industry' column
encoded_array = encoder.fit_transform(job_desc_df[['Standardized_Industry']])

#Convert the array into a DataFrame with named columns
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(['Standardized_Industry']))

#Concatenate the original DataFrame with the encoded DataFrame, without dropping anything
job_desc_df = pd.concat([job_desc_df, encoded_df], axis=1)

In [59]:
from sklearn.preprocessing import OneHotEncoder

#parse_out='False' returns a dense MumPy array
encoder = OneHotEncoder(sparse_output=False)

#Fit and transform the 'standardized_industry' column
encoded_array = encoder.fit_transform(job_desc_df[['Standardized_Department']])

#Convert the array into a DataFrame with named columns
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(['Standardized_Department']))

#Concatenate the original DataFrame with the encoded DataFrame, without dropping anything
job_desc_df = pd.concat([job_desc_df, encoded_df], axis=1)

In [61]:
#Rearranging columns to place related columns next to each other for ease of use
job_desc_df = job_desc_df[['job_id', 'title', 'location', 'department','Standardized_Department',
                           'Standardized_Department_Administrative', 'Standardized_Department_Business Consulting',
                           'Standardized_Department_Creative & Design','Standardized_Department_Customer Service',
                           'Standardized_Department_Education & Training','Standardized_Department_Engineering',
                           'Standardized_Department_Finance & Accounting','Standardized_Department_Government & Public Sector',
                           'Standardized_Department_Healthcare','Standardized_Department_Hospitality & Food Services',
                           'Standardized_Department_Human Resources', 'Standardized_Department_IT','Standardized_Department_Legal & Compliance',
                           'Standardized_Department_Logistics & Supply Chain','Standardized_Department_Manufacturing & Production',
                           'Standardized_Department_Marketing','Standardized_Department_Media & Communications','Standardized_Department_Not Provided',
                           'Standardized_Department_Operations','Standardized_Department_Product Management & Development','Standardized_Department_Project Management',
                           'Standardized_Department_Quality Assurance & Testing','Standardized_Department_Research & Development','Standardized_Department_Retail & Consumer',
                           'Standardized_Department_Sales', 'Standardized_Department_Technology', 'salary_range', 'salary_diff','company_profile', 'description', 'job_desc_length',
                           'requirements', 'benefits','telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
                           'employment_encoded',  'required_experience','experience_encoded','required_education', 'education_encoded', 'industry',
                           'Standardized_Industry', 'Standardized_Industry_Agriculture', 'Standardized_Industry_Automotive',
                           'Standardized_Industry_Construction','Standardized_Industry_Consulting','Standardized_Industry_Consumer Goods',
                           'Standardized_Industry_Education', 'Standardized_Industry_Electronics','Standardized_Industry_Engineering',
                           'Standardized_Industry_Entertainment','Standardized_Industry_Environmental Services','Standardized_Industry_Finance', 'Standardized_Industry_Government',
                           'Standardized_Industry_Health Care','Standardized_Industry_IT Services', 'Standardized_Industry_Law',
                           'Standardized_Industry_Marketing', 'Standardized_Industry_Media','Standardized_Industry_Nonprofit', 'Standardized_Industry_Not Provided',
                           'Standardized_Industry_Real Estate', 'Standardized_Industry_Retail','Standardized_Industry_Security','Standardized_Industry_Transportation',
                           'function','fraudulent']]


In [65]:
#Saving the cleaned and modified DataFrame

job_desc_df.to_csv('cleaned_fake_job_postings.csv', index=False)

In [67]:
import scipy.stats as stats
import pandas as pd
import numpy as np

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

# Select all one-hot encoded department columns (excluding the categorical 'Standardized_Department')
dept_cols = [col for col in job_desc_df.columns 
             if 'Standardized_Department_' in col 
             and col != 'Standardized_Department']

# Calculate Cramer's V between standardized department and each one-hot column
dept_results = {}
for col in dept_cols:
    dept_results[col] = cramers_v(job_desc_df['Standardized_Department'], job_desc_df[col])

# Display results sorted by strength of association
sorted_dept_results = sorted(dept_results.items(), key=lambda x: x[1], reverse=True)
print("Cramer's V between Standardized_Department and one-hot encoded columns:")
for col, v in sorted_dept_results:
    print(f"{col}: {v:.3f}")

Cramer's V between Standardized_Department and one-hot encoded columns:
Standardized_Department_Healthcare: 0.999
Standardized_Department_Business Consulting: 0.999
Standardized_Department_Customer Service: 0.999
Standardized_Department_Education & Training: 0.999
Standardized_Department_Government & Public Sector: 0.999
Standardized_Department_Product Management & Development: 0.999
Standardized_Department_Administrative: 0.999
Standardized_Department_Engineering: 0.999
Standardized_Department_Human Resources: 0.999
Standardized_Department_IT: 0.999
Standardized_Department_Manufacturing & Production: 0.999
Standardized_Department_Not Provided: 0.999
Standardized_Department_Operations: 0.999
Standardized_Department_Research & Development: 0.999
Standardized_Department_Retail & Consumer: 0.999
Standardized_Department_Sales: 0.999
Standardized_Department_Technology: 0.999
Standardized_Department_Creative & Design: 0.999
Standardized_Department_Hospitality & Food Services: 0.999
Standardi

In [69]:
import scipy.stats as stats
import pandas as pd
import numpy as np

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

# Select all one-hot encoded industry columns (excluding the categorical 'Standardized_Industry')
industry_cols = [col for col in job_desc_df.columns 
                 if 'Standardized_Industry_' in col 
                 and col != 'Standardized_Industry']

# Calculate Cramér's V between standardized industry and each one-hot column
industry_results = {}
for col in industry_cols:
    industry_results[col] = cramers_v(job_desc_df['Standardized_Industry'], job_desc_df[col])

# Display results sorted by strength of association
sorted_industry_results = sorted(industry_results.items(), key=lambda x: x[1], reverse=True)
print("Cramér's V between Standardized_Industry and one-hot encoded columns:")
for col, v in sorted_industry_results:
    print(f"{col}: {v:.3f}")


Cramér's V between Standardized_Industry and one-hot encoded columns:
Standardized_Industry_Consulting: 0.999
Standardized_Industry_Engineering: 0.999
Standardized_Industry_Environmental Services: 0.999
Standardized_Industry_Government: 0.999
Standardized_Industry_Media: 0.999
Standardized_Industry_Not Provided: 0.999
Standardized_Industry_Transportation: 0.999
Standardized_Industry_Agriculture: 0.999
Standardized_Industry_Construction: 0.999
Standardized_Industry_Consumer Goods: 0.999
Standardized_Industry_Entertainment: 0.999
Standardized_Industry_IT Services: 0.999
Standardized_Industry_Marketing: 0.999
Standardized_Industry_Nonprofit: 0.999
Standardized_Industry_Real Estate: 0.999
Standardized_Industry_Security: 0.999
Standardized_Industry_Automotive: 0.999
Standardized_Industry_Education: 0.999
Standardized_Industry_Electronics: 0.999
Standardized_Industry_Finance: 0.999
Standardized_Industry_Health Care: 0.999
Standardized_Industry_Law: 0.999
Standardized_Industry_Retail: 0.999
